# 3.0 Full MNIST Test

In [1]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)
df = {}

import clustergrammer_groupby as cby
import gene_exp_10x

In [2]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
%%time
net.load_file('../data/big_data/MNIST_row_labels.txt')
df['ini'] = net.export_df()

CPU times: user 50.4 s, sys: 1.79 s, total: 52.2 s
Wall time: 52.3 s


In [10]:
df['ini'].shape

(784, 70000)

In [4]:
df['ini'].head()

,Zero-0,Zero-1,Zero-2,Zero-3,Zero-4,Zero-5,Zero-6,Zero-7,Zero-8,Zero-9,...,Nine-6948,Nine-6949,Nine-6950,Nine-6951,Nine-6952,Nine-6953,Nine-6954,Nine-6955,Nine-6956,Nine-6957
pos_0-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Add categories

In [5]:
cols = df['ini'].columns.tolist()
new_cols = []
for inst_col in cols:
    new_col = (inst_col, 'Digit: ' + inst_col.split('-')[0])
    new_cols.append(new_col)

df['digits'] = deepcopy(df['ini'])
df['digits'].columns = new_cols

In [6]:
df['digits'].head()

,"(Zero-0, Digit: Zero)","(Zero-1, Digit: Zero)","(Zero-2, Digit: Zero)","(Zero-3, Digit: Zero)","(Zero-4, Digit: Zero)","(Zero-5, Digit: Zero)","(Zero-6, Digit: Zero)","(Zero-7, Digit: Zero)","(Zero-8, Digit: Zero)","(Zero-9, Digit: Zero)",...,"(Nine-6948, Digit: Nine)","(Nine-6949, Digit: Nine)","(Nine-6950, Digit: Nine)","(Nine-6951, Digit: Nine)","(Nine-6952, Digit: Nine)","(Nine-6953, Digit: Nine)","(Nine-6954, Digit: Nine)","(Nine-6955, Digit: Nine)","(Nine-6956, Digit: Nine)","(Nine-6957, Digit: Nine)"
pos_0-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pos_0-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Make signatures

In [7]:
%%time
pval_cutoff = 0.001
df['cat_sig'], keep_genes, keep_genes_dict = cby.generate_signatures(df['digits'], 'Digit', 
                                                                     pval_cutoff=pval_cutoff)
print(df['cat_sig'].shape)

/Users/nickfernandez/anaconda/envs/py35/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/nickfernandez/anaconda/envs/py35/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/nickfernandez/anaconda/envs/py35/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


(678, 10)
CPU times: user 5.4 s, sys: 2.67 s, total: 8.07 s
Wall time: 8.11 s


### Predict categories

In [8]:
%%time
df['pred_cat'], df['sig_sim'], df['sig_max'], y_info = cby.predict_cats_from_sigs(df['digits'], df['cat_sig'], 
                                                                                  predict_level='Predict Digit')

CPU times: user 45.6 s, sys: 945 ms, total: 46.6 s
Wall time: 46.7 s


In [9]:
%%time
df['conf'], true_count, pred_count, ser_correct, fraction_correct = cby.confusion_matrix_and_correct_series(y_info)
real_fraction_correct = deepcopy(fraction_correct)
print(real_fraction_correct)

0.814828571429
CPU times: user 168 ms, sys: 3.56 ms, total: 171 ms
Wall time: 171 ms


### Run Time
It takes about a minute to build the signature and predict the digits when we use the full MNIST dataset with 70,000 digits. 

In [14]:
100000/70000

1.4285714285714286